# Building a choropleth map

In [37]:
%matplotlib inline
import pandas as pd
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
import folium
import geocoder
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## loading the data

In [38]:
data_path = 'data/P3_GrantExport.xlsx'
grant_data = pd.read_excel(data_path)

In [39]:
grant_data.head()

,Project Number,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042,NaN


In [34]:
grant_data[['Institution','University']]

,Institution,University
0,NaN,Nicht zuteilbar - NA
1,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE
2,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO"
3,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS
4,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO"
5,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR
6,Institut für ökumenische Studien Université de...,Université de Fribourg - FR
7,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH
8,NaN,Université de Lausanne - LA
9,Laboratoire de Didactique et Epistémologie des...,Université de Genève - GE


In [29]:
geo_str = 'ch-cantons.topojson.json'

ch_map = folium.Map(location=[46.6430788,8.018626], tiles='Mapbox Bright', zoom_start=7)
ch_map.choropleth(geo_path=geo_str)
ch_map.save('ch_map.html')

In [30]:
%%HTML
<iframe width='100%' height="350" src="ch_map.html"></iframe>

In [72]:
g = geocoder.google('Geneve')
print(g.state)
print(g.country_long=='Switzerland')

GE
True


In [66]:
#grant_data['University'].apply(lambda x: geocoder.google(x))

In [73]:
for uni in grant_data['University']:
    gcoder_uni = geocoder.google(uni)
    print(uni, gcoder_uni.state, gcoder_uni.country_long)

Nicht zuteilbar - NA None None
Université de Genève - GE GE Switzerland
NPO (Biblioth., Museen, Verwalt.) - NPO None None
Universität Basel - BS BS Switzerland
NPO (Biblioth., Museen, Verwalt.) - NPO None None
Université de Fribourg - FR FR Switzerland
Université de Fribourg - FR FR Switzerland
Universität Zürich - ZH ZH Switzerland
Université de Lausanne - LA None None
Université de Genève - GE GE Switzerland
Université de Fribourg - FR FR Switzerland
NPO (Biblioth., Museen, Verwalt.) - NPO None None
Université de Genève - GE GE Switzerland
Universität Basel - BS BS Switzerland
Universität Zürich - ZH ZH Switzerland
Université de Fribourg - FR FR Switzerland
Université de Genève - GE GE Switzerland
Universität Zürich - ZH ZH Switzerland
Nicht zuteilbar - NA None None
Université de Lausanne - LA None None
Université de Genève - GE GE Switzerland
Universität Bern - BE None None
Universität Zürich - ZH None None
Université de Genève - GE None None
Nicht zuteilbar - NA None None
Nicht zut

KeyboardInterrupt: 